# 오픈소스SW개론  
## 과제 2: 시계열 기상 데이터 기반 특보 발생 탐지 판단

주어진 [Todo] 부분의 함수를 문제 조건에 맞게 완성한 후, 결과 파일 및 코드 상의 학번과 이름을 본인의 정보로 수정하여 제출하세요.

> ※ 외부 코드를 수정하여 채점 시 실행되지 않을 경우 0점 처리됩니다.  
> 반드시 `[Todo] 내부의 코드만 수정`해주시기 바랍니다.

In [83]:
import numpy as np
import pandas as pd

data_df = pd.read_csv('OBS_ASOS_TIM_20240101_20241231.csv', encoding='cp949')
data_df

,지점,지점명,일시,기온(°C),기온 QC플래그,강수량(mm),강수량 QC플래그,풍속(m/s),풍속 QC플래그,풍향(16방위),...,습도(%),습도 QC플래그,증기압(hPa),이슬점온도(°C),현지기압(hPa),현지기압 QC플래그,해면기압(hPa),해면기압 QC플래그,적설(cm),3시간신적설(cm)
0,112,인천,2024-01-01 0:00,0.7,NaN,NaN,NaN,0.6,NaN,50,...,91,NaN,5.8,-0.6,1017.7,NaN,1026.5,NaN,NaN,NaN
1,112,인천,2024-01-01 1:00,0.6,NaN,NaN,NaN,1.1,NaN,70,...,92,NaN,5.9,-0.5,1018.1,NaN,1026.9,NaN,NaN,NaN
2,112,인천,2024-01-01 2:00,0.3,NaN,NaN,NaN,1.8,NaN,360,...,93,NaN,5.8,-0.7,1018.5,NaN,1027.3,NaN,NaN,NaN
3,112,인천,2024-01-01 3:00,0.5,NaN,NaN,NaN,0.7,NaN,320,...,93,NaN,5.9,-0.5,1019.0,NaN,1027.8,NaN,NaN,NaN
4,112,인천,2024-01-01 4:00,0.4,NaN,NaN,NaN,1.5,NaN,70,...,94,NaN,5.9,-0.4,1019.2,NaN,1028.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,112,인천,2024-12-31 19:00,0.1,NaN,NaN,NaN,1.7,NaN,290,...,49,NaN,3.0,-9.3,1014.1,NaN,1022.9,NaN,NaN,NaN
8780,112,인천,2024-12-31 20:00,-0.1,NaN,NaN,NaN,2.2,NaN,270,...,53,NaN,3.2,-8.5,1014.1,NaN,1022.9,NaN,NaN,NaN
8781,112,인천,2024-12-31 21:00,0.1,NaN,NaN,NaN,2.4,NaN,270,...,56,NaN,3.5,-7.6,1014.1,NaN,1022.9,NaN,NaN,NaN
8782,112,인천,2024-12-31 22:00,0.3,NaN,NaN,NaN,2.3,NaN,290,...,59,NaN,3.7,-6.7,1014.3,NaN,1023.0,NaN,NaN,NaN


In [84]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   지점          8784 non-null   int64  
 1   지점명         8784 non-null   object 
 2   일시          8784 non-null   object 
 3   기온(°C)      8784 non-null   float64
 4   기온 QC플래그    0 non-null      float64
 5   강수량(mm)     864 non-null    float64
 6   강수량 QC플래그   1555 non-null   float64
 7   풍속(m/s)     8784 non-null   float64
 8   풍속 QC플래그    0 non-null      float64
 9   풍향(16방위)    8784 non-null   int64  
 10  풍향 QC플래그    0 non-null      float64
 11  습도(%)       8784 non-null   int64  
 12  습도 QC플래그    0 non-null      float64
 13  증기압(hPa)    8784 non-null   float64
 14  이슬점온도(°C)   8784 non-null   float64
 15  현지기압(hPa)   8784 non-null   float64
 16  현지기압 QC플래그  0 non-null      float64
 17  해면기압(hPa)   8784 non-null   float64
 18  해면기압 QC플래그  0 non-null      float64
 19  적설(cm)      273 non-null   

In [85]:
small_df = data_df[['일시', '기온(°C)', '풍속(m/s)', '습도(%)']]
small_df

,일시,기온(°C),풍속(m/s),습도(%)
0,2024-01-01 0:00,0.7,0.6,91
1,2024-01-01 1:00,0.6,1.1,92
2,2024-01-01 2:00,0.3,1.8,93
3,2024-01-01 3:00,0.5,0.7,93
4,2024-01-01 4:00,0.4,1.5,94
...,...,...,...,...
8779,2024-12-31 19:00,0.1,1.7,49
8780,2024-12-31 20:00,-0.1,2.2,53
8781,2024-12-31 21:00,0.1,2.4,56
8782,2024-12-31 22:00,0.3,2.3,59


In [86]:
def wind_alarm(df):


    df = df.copy()


    # 모든 요소가 '-'로 초기화된, (행 개수,) 형태의 numpy 배열을 생성합니다.
    wind_result = np.full(len(df), '-', dtype=object)
    wind = df['풍속(m/s)']


    wind_arr = wind.to_numpy()
    wind_next = np.empty_like(wind_arr)
    wind_next[:-1] = wind_arr[1:]
    wind_next[-1] = np.nan


    wind_result[np.isnan(wind_next)] = "판단불가"
    wind_result[(wind_next >= 21)] = "경보"
    wind_result[(wind_next >= 14) & (wind_next < 21)] = "주의보"
    wind_result[(wind_next < 14)] = "해당없음"


    return wind_result

In [87]:
def heat_alarm(df):


    # 모든 요소가 '-'로 초기화된, (행 개수,) 형태의 numpy 배열을 생성합니다.
    heat_result = np.full(len(df), '-', dtype=object)
    temperature = df['기온(°C)']


    df = df.copy()
    df['일시'] = pd.to_datetime(df['일시'])
    df = df.sort_values('일시')
    df['날짜'] = df['일시'].dt.normalize()


    max_temp_daily = temperature.groupby(df['날짜']).max()


    date_arr = df['날짜'].to_numpy()
    date_next1 = pd.to_datetime(date_arr + np.timedelta64(1, 'D')).normalize()
    date_next2 = pd.to_datetime(date_arr + np.timedelta64(2, 'D')).normalize()


    t1 = max_temp_daily.reindex(date_next1, fill_value=np.nan).to_numpy()
    t2 = max_temp_daily.reindex(date_next2, fill_value=np.nan).to_numpy()


    heat_result[np.isnan(t1) | np.isnan(t2)] = "판단불가"
    heat_result[(t1 >= 35) & (t2 >= 35)] = "경보"
    heat_result[(t1 >= 33) & (t2 >= 33) & ~((t1 >= 35) & (t2 >= 35))] = "주의보"
    heat_result[heat_result == '-'] = "해당없음"


    return heat_result



In [88]:
def dry_alarm(df):


    # 모든 요소가 '-'로 초기화된, (행 개수,) 형태의 numpy 배열을 생성합니다.
    dry_result = np.full(len(df), '-', dtype=object)
    humidity = df['습도(%)']


    df = df.copy()
    df['일시'] = pd.to_datetime(df['일시'])
    df = df.sort_values('일시')
    df['날짜'] = df['일시'].dt.date


    humidity = humidity.reset_index(drop=True)
    humidity.index = df.index


    avg_humidity_daily = humidity.groupby(df['날짜']).mean()


    effective_humidity = (0.7 * avg_humidity_daily + 0.2 * avg_humidity_daily.shift(1) + 0.1 * avg_humidity_daily.shift(2))


    date_arr = df['날짜'].to_numpy()
    date_next1 = pd.to_datetime(date_arr) + pd.Timedelta(days=1)
    date_next2 = pd.to_datetime(date_arr) + pd.Timedelta(days=2)


    eff_hum1 = effective_humidity.reindex(date_next1.date, fill_value=np.nan).to_numpy()
    eff_hum2 = effective_humidity.reindex(date_next2.date, fill_value=np.nan).to_numpy()


    dates = sorted(avg_humidity_daily.index)
    date_series = pd.Series(date_arr)
    is_first_2days = date_series.isin(dates[:2])
    is_last_2days = date_series.isin(dates[-2:])
    is_nan = np.isnan(eff_hum1) | np.isnan(eff_hum2)


    dry_result[is_first_2days | is_last_2days | is_nan] = "판단불가"
    dry_result[(eff_hum1 <= 25) & (eff_hum2 <= 25) & (dry_result == '-')] = "경보"
    dry_result[(eff_hum1 <= 35) & (eff_hum2 <= 35) & (dry_result == '-')] = "주의보"
    dry_result[dry_result == '-'] = "해당없음"


    return dry_result



In [89]:
def cold_alarm(df):


    # 모든 요소가 '-'로 초기화된, (행 개수,) 형태의 numpy 배열을 생성합니다.
    cold_result = np.full(len(df), '-', dtype=object)
    temperature = df['기온(°C)']


    df = df.copy()
    df['일시'] = pd.to_datetime(df['일시'])
    df = df.sort_values('일시')
    df['날짜'] = df['일시'].dt.date
    df['시간'] = df['일시'].dt.hour


    temperature = temperature.reset_index(drop=True)
    temperature.index = df.index


    temp_morning = df[(df['시간'] >= 3) & (df['시간'] < 9)].copy()
    temp_morning['기온(°C)'] = temperature[temp_morning.index]
    temp_morning_min = temp_morning.groupby('날짜')['기온(°C)'].min()


    date_arr = df['날짜'].to_numpy()
    date_next1 = pd.to_datetime(date_arr) + pd.Timedelta(days=1)
    date_next2 = pd.to_datetime(date_arr) + pd.Timedelta(days=2)


    min_today = temp_morning_min.reindex(date_arr, fill_value=np.nan).to_numpy()
    min_next1 = temp_morning_min.reindex(date_next1.date, fill_value=np.nan).to_numpy()
    min_next2 = temp_morning_min.reindex(date_next2.date, fill_value=np.nan).to_numpy()


    valid_month = df['일시'].dt.month.isin([10, 11, 12, 1, 2, 3, 4])
    is_nan = np.isnan(min_today) | np.isnan(min_next1) | np.isnan(min_next2)


    warn_drop_cond = (min_next1 <= min_today - 15) & (min_next1 <= 3)
    warn_low_cond = (min_next1 <= -15) & (min_next2 <= -15)


    adv_drop_cond = (min_next1 <= min_today - 10) & (min_next1 <= 3)
    adv_low_cond = (min_next1 <= -12) & (min_next2 <= -12)


    cold_result[is_nan | ~valid_month] = "판단불가"
    cold_result[(warn_drop_cond | warn_low_cond) & (cold_result == '-')] = "경보"
    cold_result[(adv_drop_cond | adv_low_cond) & (cold_result == '-')] = "주의보"
    cold_result[cold_result == '-'] = "해당없음"


    return cold_result




## 검증코드

아래 검증 데이터를 통과한 매소드에 대해서만 구현이 인정됩니다.

검증 코드를 통과 후 최종 csv 파일을 제출해주세요.

In [90]:
def evaluate_prediction(pred, label, label_name):
    correct = np.sum(pred == label)
    total = len(label)
    accuracy = correct / total
    print(f'[{label_name:<5}] 정답 확인 결과: ({correct:>3}/{total:<3}) → 정확도: {accuracy:.2%}')



def validate_wind_alarm():
    df = pd.read_csv('val_data.csv', encoding='cp949')
    gt = df['강풍 특보'].values
    pred = wind_alarm(df)
    evaluate_prediction(np.array(pred), gt, '강풍 특보')

def validate_heat_alarm():
    df = pd.read_csv('val_data.csv', encoding='cp949')
    gt = df['폭염 특보'].values
    pred = heat_alarm(df)
    evaluate_prediction(np.array(pred), gt, '폭염 특보')

def validate_dry_alarm():
    df = pd.read_csv('val_data.csv', encoding='cp949')
    gt = df['건조 특보'].values
    pred = dry_alarm(df)
    evaluate_prediction(np.array(pred), gt, '건조 특보')

def validate_cold_alarm():
    df = pd.read_csv('val_data.csv', encoding='cp949')
    gt = df['한파 특보'].values
    pred = cold_alarm(df)
    evaluate_prediction(np.array(pred), gt, '한파 특보')

def validate_all():
    print('\n[📊 특보 검증 결과 요약]\n' + '-'*40)
    validate_wind_alarm()
    validate_heat_alarm()
    validate_dry_alarm()
    validate_cold_alarm()
    print('-'*40)

validate_all()


[📊 특보 검증 결과 요약]
----------------------------------------
[강풍 특보] 정답 확인 결과: (168/168) → 정확도: 100.00%
[폭염 특보] 정답 확인 결과: (168/168) → 정확도: 100.00%
[건조 특보] 정답 확인 결과: (168/168) → 정확도: 100.00%
[한파 특보] 정답 확인 결과: (168/168) → 정확도: 100.00%
----------------------------------------


In [91]:
final_df = small_df.copy()
final_df['강풍 특보'] = wind_alarm(small_df)
final_df['건조 특보'] = dry_alarm(small_df)
final_df['폭염 특보'] = heat_alarm(small_df)
final_df['한파 특보'] = cold_alarm(small_df)

final_df.to_csv('12214206_한상민_특보결과.csv', index=False, encoding='cp949')

In [92]:
final_df

,일시,기온(°C),풍속(m/s),습도(%),강풍 특보,건조 특보,폭염 특보,한파 특보
0,2024-01-01 0:00,0.7,0.6,91,해당없음,판단불가,해당없음,해당없음
1,2024-01-01 1:00,0.6,1.1,92,해당없음,판단불가,해당없음,해당없음
2,2024-01-01 2:00,0.3,1.8,93,해당없음,판단불가,해당없음,해당없음
3,2024-01-01 3:00,0.5,0.7,93,해당없음,판단불가,해당없음,해당없음
4,2024-01-01 4:00,0.4,1.5,94,해당없음,판단불가,해당없음,해당없음
...,...,...,...,...,...,...,...,...
8779,2024-12-31 19:00,0.1,1.7,49,해당없음,판단불가,판단불가,판단불가
8780,2024-12-31 20:00,-0.1,2.2,53,해당없음,판단불가,판단불가,판단불가
8781,2024-12-31 21:00,0.1,2.4,56,해당없음,판단불가,판단불가,판단불가
8782,2024-12-31 22:00,0.3,2.3,59,해당없음,판단불가,판단불가,판단불가
